# Framework
Test for the framework with which we will compare the different distance measurements.

In [260]:
# import all necessary libraries
import distance

from similarity.levenshtein import Levenshtein
levenshtein = Levenshtein()

from similarity.damerau import Damerau
damerau = Damerau()

from pyjarowinkler import distance as jwdistance
from similarity.jarowinkler import JaroWinkler
jarowinkler = JaroWinkler()

from similarity.weighted_levenshtein import WeightedLevenshtein
from similarity.weighted_levenshtein import CharacterSubstitutionInterface
import math
class CharacterSubstitution(CharacterSubstitutionInterface):
    def cost(self, c0, c1):
        return math.inf # assign inifte weight to all substitutions
levenshtein2 = WeightedLevenshtein(CharacterSubstitution())

import numpy as np
from datetime import datetime

In [261]:
import random
import string

def get_random_string(length):
    #letters = ["a", "b", "c", "d", "e"] # limited alphabet to produce duplicates
    letters = string.ascii_lowercase # extended alphabet
    result_str = ''.join(random.choice(letters) for i in range(length))
    return result_str

In [268]:
# dummy data structure
n = 300 # number of strings
l = 10 # length of strings

strings = []
for i in range(n):
    strings.append(get_random_string(l))
#strings = ["Sarah", "Raphael", "Sushit", "Max", "Jan", "Micky", "Caro", "Bekir", "Fabian", "Ossi"]

print("First 5 strings:", strings[:5])

n = len(strings)
distMatrix = np.full((n, n), 0)

First 5 strings: ['ilbulggphq', 'zfzbfeuoew', 'kybbyhimkj', 'foysyllylq', 'cpcrktpbix']


## 1. Naive implementation

In [286]:
start = datetime.now()
distMatrix = np.full((n, n), 0)

# iterate through every combination and calculate distance
for i, x in enumerate(strings):
    for j, y in enumerate(strings):
        #dist = distance.hamming(x, y) # Hamming distance - TODO: adjustment for length
        dist = distance.levenshtein(x, y) # Levenshtein v1
        #dist = levenshtein.distance(x, y) # Levenshtein v2 (faster than v1)
        #dist = levenshtein2.distance(x, y) # Levenshtein II
        #dist = damerau.distance(x, y) # Damerau-Levenshtein
        #dist = jwdistance.get_jaro_distance(x, y) # Jaro-Winkler v1
        #dist = jarowinkler.similarity(x, y) # Jaro-Winkler v2 (faster than v1)
        distMatrix[i][j] = dist

end = datetime.now()

print("Time needed:", end-start)
print(distMatrix)

Iterations: 0
Iterations: 100
Iterations: 200
Time needed: 0:00:07.370834
[[ 0  0  0 ...  0  0  0]
 [10  0  9 ... 10  9  9]
 [ 9  9  0 ... 10 10 10]
 ...
 [ 9 10 10 ...  0  8  9]
 [10  9 10 ...  8  0 10]
 [10  9 10 ...  9 10  0]]


## 2. Upper left triangle
Hypothesis: Actual distance calculations take up the most time. Iteration overhead it negligable. Matrix mirroring is negligable.

In [285]:
start = datetime.now()
distMatrix = np.full((n, n), 0)

for i, x in enumerate(strings):
    for j, y in enumerate(strings):
        if j > i: # only calculate upper right triangle of matrix
            #dist = distance.hamming(x, y) # Hamming distance - TODO: adjustment for length
            dist = distance.levenshtein(x, y) # Levenshtein v1
            #dist = levenshtein.distance(x, y) # Levenshtein v2 (faster than v1)
            #dist = levenshtein2.distance(x, y) # Levenshtein II
            #dist = damerau.distance(x, y) # Damerau-Levenshtein
            #dist = jwdistance.get_jaro_distance(x, y) # Jaro-Winkler v1
            #dist = jarowinkler.similarity(x, y) # Jaro-Winkler v2 (faster than v1)

            distMatrix[i][j] = dist

# mirror upper right triangle of matrix by adding the transposition and subratcting the diagonal
distMatrix = distMatrix + distMatrix.T - np.diag(np.diag(distMatrix)) 

end = datetime.now()

print("Time needed:", end-start)
print(distMatrix)

Iterations: 0
Iterations: 100
Iterations: 200
Time needed: 0:00:03.809895
[[ 0  0  0 ...  0  0  0]
 [ 0  0  9 ... 10  9  9]
 [ 0  9  0 ... 10 10 10]
 ...
 [ 0 10 10 ...  0  8  9]
 [ 0  9 10 ...  8  0 10]
 [ 0  9 10 ...  9 10  0]]


## 3. Dictionary Lookup
Hypothesis: Lookup of already calculated values is faster than recalculation. There is a significant number of duplicates in the process log when considering all traces and not only variants. Adjust alphabet of `get_random_string()` to try.

In [287]:
start = datetime.now()
distMatrix = np.full((n, n), 0)
d = {}

# iterate through every combination and calculate distance
for i, x in enumerate(strings):
    for j, y in enumerate(strings):
        if j > i: # only calculate upper right triangle of matrix
            if  (x, y) in d.keys():
                dist = d[(x, y)]
            else:
                #dist = distance.hamming(x, y) # Hamming distance - TODO: adjustment for length
                dist = distance.levenshtein(x, y) # Levenshtein v1
                #dist = levenshtein.distance(x, y) # Levenshtein v2 (faster than v1)
                #dist = levenshtein2.distance(x, y) # Levenshtein II
                #dist = damerau.distance(x, y) # Damerau-Levenshtein
                #dist = jwdistance.get_jaro_distance(x, y) # Jaro-Winkler v1
                #dist = jarowinkler.similarity(x, y) # Jaro-Winkler v2 (faster than v1)
                
                # add calculated distance to dictionary
                d[(x, y)] = dist
                d[(y, x)] = dist

            distMatrix[i][j] = dist

# mirror upper right triangle of matrix by adding the transposition and subratcting the diagonal
distMatrix = distMatrix + distMatrix.T - np.diag(np.diag(distMatrix)) 

end = datetime.now()


print("Time needed:", end-start)
print(distMatrix)

Time needed: 0:00:04.469166
[[ 0 10  9 ...  9 10 10]
 [10  0  9 ... 10  9  9]
 [ 9  9  0 ... 10 10 10]
 ...
 [ 9 10 10 ...  0  8  9]
 [10  9 10 ...  8  0 10]
 [10  9 10 ...  9 10  0]]
